[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Pricing_Seguros_con_RedesNeuronales/blob/main/notebooks/Introduccion_Tensorflow_Keras_Seguros.ipynb)

# Introducción a Tensorflow/Keras

In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.3 MB/s eta 0:00:00


In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tf_keras


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

import os

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio

# Creación de datos sintéticos

In [ ]:
# Definir el rango y la resolución
delta = 0.1
x = np.arange(-2, 2 + delta, delta)
y = np.arange(-2, 2 + delta, delta)
X, Y = np.meshgrid(x, y)

# Definir la función de la superficie
Z = np.sin(X**2 + Y**2)

# Convertir a un DataFrame para facilitar el uso en redes neuronales
df = pd.DataFrame({
    'x': X.ravel(),
    'y': Y.ravel(),
    'z': Z.ravel()
})


In [ ]:
df

,x,y,z
0,-2.0,-2.0,0.989358
1,-1.9,-2.0,0.970384
2,-1.8,-2.0,0.817361
3,-1.7,-2.0,0.570254
4,-1.6,-2.0,0.273293
...,...,...,...
1676,1.6,2.0,0.273293
1677,1.7,2.0,0.570254
1678,1.8,2.0,0.817361
1679,1.9,2.0,0.970384


In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.update_layout(title='Superficie 3D', scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
fig.show()

# Separación de datos para entrenamiento y pruebas

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['x', 'y']], df['z'], test_size=0.2, random_state=1)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=X_test['x'],
    y=X_test['y'],
    z=y_test,
    mode='markers',  # 'markers' para puntos, 'lines' para líneas
    marker=dict(size=2, color=df['z'], colorscale='Viridis', opacity=0.8)
)])

# Configurar la visualización
fig.update_layout(title='Puntos 3D de la Superficie',
                  scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

# Mostrar la figura
fig.show()

# Creación y entrenamiento de la red

In [ ]:
# Definir el modelo de la red neuronal
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(2,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Capa de salida
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [ ]:
# Compilar el modelo usando el optimizador Adam y la función de pérdida MSE
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])


In [ ]:

# Crear carpeta para almacenar imágenes
output_dir = "training_frames"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
class MonitorCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        predictions = self.model.predict(X_test, verbose=0)

        fig = go.Figure(data=[go.Scatter3d(
            x=X_test['x'],
            y=X_test['y'],
            z=predictions.flatten(),
            mode='markers',
            marker=dict(size=2, color=predictions.flatten(), colorscale='Viridis', opacity=0.8)
        )])

        fig.update_layout(title=f"Predicciones antes de la época {epoch+1}",
                          scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z (predicho)"))

        fig.write_html(f"training_frames/epoch_{epoch+1}.html")

    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Error Absoluto Medio (entrenamiento) = {logs['mae']:.4f}")


In [ ]:
# Entrenar la red neuronal
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[MonitorCallback()], verbose=0)


Epoch 1: Error Absoluto Medio (entrenamiento) = 0.5349
Epoch 2: Error Absoluto Medio (entrenamiento) = 0.3798
Epoch 3: Error Absoluto Medio (entrenamiento) = 0.2513
Epoch 4: Error Absoluto Medio (entrenamiento) = 0.2101
Epoch 5: Error Absoluto Medio (entrenamiento) = 0.1899
Epoch 6: Error Absoluto Medio (entrenamiento) = 0.1542
Epoch 7: Error Absoluto Medio (entrenamiento) = 0.1653
Epoch 8: Error Absoluto Medio (entrenamiento) = 0.1572
Epoch 9: Error Absoluto Medio (entrenamiento) = 0.1434
Epoch 10: Error Absoluto Medio (entrenamiento) = 0.1188
Epoch 11: Error Absoluto Medio (entrenamiento) = 0.1126
Epoch 12: Error Absoluto Medio (entrenamiento) = 0.1555
Epoch 13: Error Absoluto Medio (entrenamiento) = 0.0668
Epoch 14: Error Absoluto Medio (entrenamiento) = 0.0546
Epoch 15: Error Absoluto Medio (entrenamiento) = 0.0644
Epoch 16: Error Absoluto Medio (entrenamiento) = 0.0453
Epoch 17: Error Absoluto Medio (entrenamiento) = 0.0606
Epoch 18: Error Absoluto Medio (entrenamiento) = 0.0881
E

# Evaluemos el modelo

In [ ]:
# Definir el rango y la resolución
delta = 0.1
x = np.arange(-2, 2 + delta, delta)
y = np.arange(-2, 2 + delta, delta)
X, Y = np.meshgrid(x, y)

# Definir la función de la superficie
Z = model.predict(np.c_[X.ravel(), Y.ravel()])

# Convertir a un DataFrame para facilitar el uso en redes neuronales
df = pd.DataFrame({
    'x': X.ravel(),
    'y': Y.ravel(),
    'z': Z.ravel()
})


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',  # 'markers' para puntos, 'lines' para líneas
    marker=dict(size=2, color=df['z'], colorscale='Viridis', opacity=0.8)
)])

# Configurar la visualización
fig.update_layout(title='Puntos 3D de la Superficie',
                  scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

# Mostrar la figura
fig.show()

In [ ]:
# Extract unique x and y values
x_unique = np.sort(df['x'].unique())
y_unique = np.sort(df['y'].unique())

# Create a meshgrid
X, Y = np.meshgrid(x_unique, y_unique)

# Pivot the DataFrame to get Z values in a grid format
Z = df.pivot(index='y', columns='x', values='z').values

# Create the surface plot
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.update_layout(title='Superficie 3D', scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))

fig.show()

In [ ]:
Z = model.predict(X_test)
Z_real = np.sin(X_test['x']**2 + X_test['y']**2)
# Convertir a un DataFrame para facilitar el uso en redes neuronales
df = pd.DataFrame({
    'x': X_test['x'].ravel(),
    'y': X_test['y'].ravel(),
    'z': Z.ravel(),
    'z_real': Z_real.ravel()
})

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


<ipython-input-20-4ae51f690bb7>:5: FutureWarning:

Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.

<ipython-input-20-4ae51f690bb7>:6: FutureWarning:

Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.

<ipython-input-20-4ae51f690bb7>:8: FutureWarning:

Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.



In [ ]:
df

,x,y,z,z_real
0,0.3,1.2,0.964652,0.999168
1,0.8,1.2,0.875561,0.873133
2,-1.9,-1.3,-0.808871,-0.832267
3,-0.6,-1.3,0.834555,0.887362
4,1.4,2.0,-0.400142,-0.317589
...,...,...,...,...
332,0.3,1.7,0.145532,0.160890
333,0.6,0.3,0.424840,0.434966
334,-0.8,-1.7,-0.447286,-0.378715
335,-1.1,0.1,0.943031,0.939099


In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
# Predict the values for the test set
y_pred_train = model.predict(X_train)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_train, y_pred_train)
mae

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.03400829430692942

In [ ]:
# Predict the values for the test set
y_pred = model.predict(X_test)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
mae

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


0.034932396735437556

# ¿Porque todo parece tan perfecto?

- Los datos fueron escogidos sintéticamente, no son de un experimento real.

- En la muestra, NO EXISTEN 2 valores que tengan características X muy parecidas pero que su valores Y sea diferente, cosa que generalmente pasa en la realidad, ¿que significa esto?

- Evaluamos valores en los que el modelo tenía datos, ¿Pero que pasa fuera de estos? Veamos la imagen de abajo


In [ ]:
# Definir el rango y la resolución
delta = 0.1
x = np.arange(-5, 5 + delta, delta)
y = np.arange(-5, 5 + delta, delta)
X, Y = np.meshgrid(x, y)

# Definir la función de la superficie
Z = model.predict(np.c_[X.ravel(), Y.ravel()])
Z_real = np.sin(X**2 + Y**2)
# Convertir a un DataFrame para facilitar el uso en redes neuronales
df = pd.DataFrame({
    'x': X.ravel(),
    'y': Y.ravel(),
    'z': Z.ravel(),
    'z_real': Z_real.ravel()
})

319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
df

,x,y,z,z_real
0,-5.0,-5.0,13.454725,-0.262375
1,-4.9,-5.0,13.247635,-0.950699
2,-4.8,-5.0,13.040543,-0.793239
3,-4.7,-5.0,12.838976,0.033883
4,-4.6,-5.0,12.658468,0.821416
...,...,...,...,...
10196,4.6,5.0,11.499626,0.821416
10197,4.7,5.0,11.670832,0.033883
10198,4.8,5.0,11.842041,-0.793239
10199,4.9,5.0,12.013246,-0.950699


In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',  # 'markers' para puntos, 'lines' para líneas
    marker=dict(size=2, color=df['z'], colorscale='Viridis', opacity=0.8)
)])

# Configurar la visualización
fig.update_layout(title='Puntos 3D de la Superficie',
                  scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

# Mostrar la figura
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z_real'],
    mode='markers',  # 'markers' para puntos, 'lines' para líneas
    marker=dict(size=2, color=df['z_real'], colorscale='Viridis', opacity=0.8)
)])

# Configurar la visualización
fig.update_layout(title='Puntos 3D de la Superficie',
                  scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

# Mostrar la figura
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z_real, x=X, y=Y)])
fig.update_layout(title='Superficie 3D', scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
fig.show()